- [Reference](https://nuxlear.tistory.com/4)

# Deprecated

In [ ]:
# import h5py
# import numpy as np
# 
# filename = 'test.hdf5'
# with h5py.File(filename, 'w') as f:
#     # 1000개의 32x32 RGB 이미지를 담는 데이터 공간을 생성
#     f.create_dataset('image', (1000, 32, 32, 3), dtype='float32')
#     # 1000개의 float을 담는 데이터 공간을 생성
#     f.create_dataset('label', (1000,), dtype='float32')
#     image_set = f['image'] # 실 데이터 공간에 접근할 변수를 할당
#     label_set = f['label']
#     
#     # 파일에 값을 대입
#     label_set[0] = 3.0 # dataset에 값을 대입
#     label_set[10:20] = np.arange(10.) # 배열을 직접 대입
#     labels = label_set[:] # 이미지 데이터 전체를 반환하지만, 메모리에 올리지는 않는다. 값에 접근할 때 메모리에 올린다.
#     print(labels)
# 
#     array = label_set[:50] # 데이터셋을 numpy array 와 비슷한 형태로 불러올 수 있다. 역시 이 명령으로 메모리에 올라가지는 않는다. 

# from keras.utils.io_utils import HDF5Matrix
#  
# n_epoch = 1000
# batch_size = 32
# split_pos = 800
#  
# x_data = HDF5Matrix(filename, 'image')    # 위에서 생성한 HDF5 파일의 image 경로의 데이터를 가져오게 된다. 
# x_train = HDF5Matrix(filename, 'image', end=split_pos)    # HDF5 파일의 데이터 중 일부만 가져오는 것도 가능하다. 
# x_test = HDF5Matrix(filename, 'image', start=split_pos)
# y_train = HDF5Matrix(filename, 'label', end=split_pos)
# y_test = HDF5Matrix(filename, 'label', start=split_pos)

# HDF5 파일 생성 및 데이터 저장

In [1]:
import h5py
import numpy as np

filename = 'test.hdf5'

# HDF5 파일 생성 및 데이터 저장
with h5py.File(filename, 'w') as f:
    # 1000개의 32x32 RGB 이미지를 담는 데이터 공간을 생성
    f.create_dataset('image', (1000, 32, 32, 3), dtype='float32')
    # 1000개의 float을 담는 데이터 공간을 생성
    f.create_dataset('label', (1000,), dtype='float32')
    image_set = f['image'] # 실 데이터 공간에 접근할 변수를 할당
    label_set = f['label']
    
    # 이미지와 라벨에 임의의 값을 대입
    image_set[:] = np.random.rand(1000, 32, 32, 3)
    label_set[:] = np.random.randint(0, 10, size=(1000,))

# 데이터 로드 및 분할

In [2]:
def loda_data_and_split(filename, image_key='image', label_key='label', start=None, end=None):
    with h5py.File(filename, 'r') as f:
        image_set = f[image_key][start:end]
        label_set = f[label_key][start:end]
    
    return image_set, label_set

split_pos = 800

# 학습 데이터 로드
x_train, y_train = loda_data_and_split(filename, start=0, end=split_pos)

# 테스트 데이터 로드
x_test, y_test = loda_data_and_split(filename, start=split_pos)

# Tensorflow 데이터셋 생성

In [4]:
import tensorflow as tf

def create_tf_dataset(images, labels):
    return tf.data.Dataset.from_tensor_slices((images, labels))

# TensorFlow 데이터셋 생성
train_dataset = create_tf_dataset(x_train, y_train)
test_dataset = create_tf_dataset(x_test, y_test)

batch_size = 32
buffer_size= 1024
train_dataset = train_dataset.shuffle(buffer_size=buffer_size).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

### shuffle(buffersize).batch
    - buffer_size는 한 번에 섞는 데이터의 크기
    - buffer_size=1024, batch_size = 32
        - 데이터셋에서 최대 1024개의 요소가 버퍼에 로드되고 이 버퍼 내에서 데이터가 무작위로 섞임.
        - 그 후, 섞인 데이터가 배치로 생성

# Build Model

In [5]:
# 모델 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
21/25 [========================>.....] - ETA: 0s - loss: 2.3829 - accuracy: 0.0818

2024-05-28 21:16:16.382403: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [800,32,32,3]
	 [[{{node Placeholder/_0}}]]
2024-05-28 21:16:16.382518: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [800]
	 [[{{node Placeholder/_1}}]]
2024-05-28 21:16:16.469879: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-05-28 21:16:16.562644: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this

25/25 [==============================] - 0s 5ms/step - loss: 2.3773 - accuracy: 0.0763 - val_loss: 2.3306 - val_accuracy: 0.0900
Epoch 2/10
25/25 [==============================] - 0s 3ms/step - loss: 2.2840 - accuracy: 0.1250 - val_loss: 2.3012 - val_accuracy: 0.1300
Epoch 3/10
25/25 [==============================] - 0s 3ms/step - loss: 2.2201 - accuracy: 0.2275 - val_loss: 2.3186 - val_accuracy: 0.0650
Epoch 4/10
25/25 [==============================] - 0s 3ms/step - loss: 2.1165 - accuracy: 0.3088 - val_loss: 2.3202 - val_accuracy: 0.0800
Epoch 5/10
25/25 [==============================] - 0s 3ms/step - loss: 1.9807 - accuracy: 0.4038 - val_loss: 2.3264 - val_accuracy: 0.1050
Epoch 6/10
25/25 [==============================] - 0s 3ms/step - loss: 1.8120 - accuracy: 0.6587 - val_loss: 2.3338 - val_accuracy: 0.1200
Epoch 7/10
25/25 [==============================] - 0s 3ms/step - loss: 1.6138 - accuracy: 0.7200 - val_loss: 2.3473 - val_accuracy: 0.0850
Epoch 8/10
25/25 [=============